In [1]:
import numpy as np
import tnn
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.nn.functional as f

from datasets import load_dataset 
from torch.utils.data import DataLoader

In [2]:
dataset = load_dataset("ylecun/mnist", num_proc=2)
train = dataset.get("train")
test = dataset.get("test")

In [3]:
def to_numpy(example):
    arr = np.reshape(example["image"], -1) / 255.0
    example["input"] = arr
    return example

train_dataset = train.map(to_numpy, num_proc=2).select_columns(["input", "label"])
test_dataset = test.map(to_numpy, num_proc=2).select_columns(["input", "label"])

In [4]:
def collate_fn(batch):
    inputs = torch.tensor([ex["input"] for ex in batch]).float()
    labels = torch.tensor([ex["label"] for ex in batch]).long()
    return inputs, labels

trainloader = data.DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=False, collate_fn=collate_fn, num_workers=2)
testloader = data.DataLoader(test_dataset, batch_size=1024, shuffle=False, drop_last=False, collate_fn=collate_fn, num_workers=2)

In [5]:
class MLP(nn.Module):

    def __init__(self):
        super().__init__()
        self.linear_1 = nn.Linear(784, 512)
        self.drop_1 = nn.Dropout(0.5)
        self.linear_2 = nn.Linear(512, 512)
        self.drop_2 = nn.Dropout(0.25)
        self.linear_3 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.drop_1(f.relu(self.linear_1(x)))
        x = self.drop_2(f.relu(self.linear_2(x)))
        return dict(logits=f.relu(self.linear_3(x)))

In [6]:
model = tnn.Model(MLP())
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), momentum=0.9)

In [7]:
trainer = tnn.Trainer(model, optim, loss_fn, trainloader, testloader, path="./train.h5", verbose=True)

In [8]:
metrics = trainer.train(epochs=3)

model using cuda
weights saved to ./train.h5/trajectory/weights-epoch-0
training started
(epoch: 1): (train loss: 2.1982, test loss: 1.9430, train acc: 0.3731, test acc: 0.6495)
weights saved to ./train.h5/trajectory/weights-epoch-1
(epoch: 2): (train loss: 1.4416, test loss: 0.8442, train acc: 0.6577, test acc: 0.7927)
weights saved to ./train.h5/trajectory/weights-epoch-2
(epoch: 3): (train loss: 0.7757, test loss: 0.5354, train acc: 0.7730, test acc: 0.8568)
weights saved to ./train.h5/trajectory/weights-epoch-3
training complete
train_losses saved to ./train.h5/metrics/train_losses
test_losses saved to ./train.h5/metrics/test_losses
train_accs saved to ./train.h5/metrics/train_accs
test_accs saved to ./train.h5/metrics/test_accs


In [9]:
model = tnn.Model(MLP())
landscape = tnn.Landscape.from_file("./train.h5", model, loss_fn, testloader, device="cuda", path="./train.h5", verbose=True)

In [10]:
data = landscape.create_meshgrid(resolution=25, endpoints=(-10.0, 10.0), mode="pca")

meshgrid creation using pca
model using cuda
meshgrid creation started
(iter: 1): iter loss: 4.6193
(iter: 2): iter loss: 2.3046
(iter: 3): iter loss: 3.1580
(iter: 4): iter loss: 23.9986
(iter: 5): iter loss: 0.8442
(iter: 6): iter loss: 26.8062
(iter: 7): iter loss: 104.3009
(iter: 8): iter loss: 4.6207
(iter: 9): iter loss: 126.8287
meshgrid creation complete
meshgrid saved to ./train.h5/landscape/meshgrid
optim_path saved to ./train.h5/landscape/optim_path
directions saved to ./train.h5/landscape/directions
variance saved to ./train.h5/landscape/variance


In [11]:
path = "./train.h5"
metrics = tnn.get_metrics_from_file(path)

In [12]:
trajectory = tnn.get_trajectory_from_file(path)

In [13]:
meshgrid = tnn.get_meshgrid_from_file(path)

In [14]:
landscape = tnn.get_landscape_from_file(path)